In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

## Se definen las configuraciones

In [ ]:
l_rates = [0.001, 0.01]
epochs = [10,20,30]
batch_sizes = [8,16,30]

In [ ]:
experiment = 1 #Número donde inician los experimentos
for batch in batch_sizes:
    for epoch in epochs:
        for learning_rate in l_rates:
        
            print("*"*20)
            print("EXPERIMENTO: Batch:", batch, "Epoc: ", epoch, "learning_rate: ", learning_rate)
            
            #Cargar los parametros cada que inicia un experimento
            params = utils.yaml_to_dict(os.path.join('..','config.yml'))
            params['data_dir'] = os.path.join('..',params['data_dir'])
            
            experiment_name = 'Experimento10-Configuracion'+str(experiment)
            params['model_dir'] = os.path.join('..','experiments-weights',experiment_name)
            params['batch_size'] = batch
            params['num_epochs'] = epoch
            params['learning_rate'] = learning_rate
                        
            width, height = params["image_shape"]
            
            #Limpiar Sesión
            tf.keras.backend.clear_session()
            
            #Cargar el modelo
            inputs = tf.keras.layers.Input(shape=(width,height,3))
            net = model.ModelArchitecture(num_classes=params['num_classes'])
            x = net(inputs, training=False)
            
            #Generar conjuntos de entrenamiento y validación y llevarlos a un generador
            preprocessing.split_data(params)
            train_generator = data.make_datagenerator(params, mode='training')
            val_generator = data.make_datagenerator(params,mode='validation')
            
            #Crear el directorio que guarda los checkpoints
            if not os.path.exists(params['model_dir']):
                os.makedirs(params['model_dir'])

            #Definir los callbacks
            cp_callback = tf.keras.callbacks.ModelCheckpoint(
                os.path.join(params['model_dir'], 'tf_ckpt'), 
                save_weights_only=True, 
                verbose=1,
                period=5)

            tb_callback = tf.keras.callbacks.TensorBoard(
                os.path.join(params['model_dir'], 'logs'))

            #Definir la función de optimización
            optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

            #Definir los pasos para entrenamiento y validación
            steps_per_epoch = train_generator.n // params['batch_size']
            validation_steps = val_generator.n // params['batch_size']

            #Entrenar el modelo
            net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
            net.fit_generator(
                train_generator, 
                steps_per_epoch=steps_per_epoch, 
                epochs=params['num_epochs'],
                workers=4,
                validation_data=val_generator, 
                validation_steps=validation_steps,
                callbacks=[cp_callback,tb_callback])
            
    
            #Obtener las métricas de resultados en el conjunto de validación
            params['batch_size'] = val_generator.n
            params['shuffle'] = False
            val_generator = data.make_datagenerator(params,mode='validation')
            Y_pred = net.predict_generator(val_generator, steps=1)
            y_pred = np.argmax(Y_pred, axis=1)
            
            print(confusion_matrix(val_generator.data, y_pred))
            target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
            print(classification_report(val_generator.data, y_pred, target_names=target_names))
            
            #Aumentar el contador de experimentos
            experiment += 1